In [1]:
import pandas as pd

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['No'].tolist()

总数量：2


[1, 2]

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

url = input_.loc[a, 'Url']

dict_param = {}
list_param = url.split('?')[1].split('&') if '?' in url else []
for param in list_param:
    dict_param[param.split('=')[0]] = param.split('=')[1]

dict_param.pop('start', '')
dict_param.pop('num', '')

url = url.split('?')[0] + '?'
for key, value in dict_param.items():
    url += f'{key}={value}&'

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.get(url[:-1],
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))
        
        if resp.status_code == 200:
            break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1


<Response [200]>

In [6]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(resp.text, 'lxml')
html = etree.HTML(str(soup))

with open('./page.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <title>
   Dorman Products - Search Results
  </title>
  <meta charset="utf-8"/>
  <meta content="no-cache, no-store, must-revalidate" http-equiv="cache-control"/>
  <meta content="no-cache" http-equiv="pragma"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="-1" http-equiv="expires"/>
  <meta content="Dorman gives auto repair professionals and vehicle owners greater freedom to fix cars and trucks by focusing on solutions first." name="description"/>
  <meta content="auto parts, car parts, automotive parts, aftermarket car parts, automotive repair parts, window regulator, control arm, door handle, fluid reservoir, remanufactured auto parts, intake manifold, exhaust manifold, diesel automotive part, instrument cluster, automotive electronic parts, clock spring, engine car part, chassis, car part, car mirror, transmission parts" name="keywords"/>
  <meta content="width=device-width, initial-scale=1.0, max

In [7]:
count = int(html.xpath('//span[@id="lblResultCount"]/text()')[0].strip())

count

1212

In [8]:
count <= 0

False

In [9]:
import math

output = pd.DataFrame([{'No': input_.loc[a, 'No'],
                        'Url': url[:-1],
                        'SKU Count': count,
                        'Page': math.ceil(count / 100)}])

output = output.sort_values(by=['No'],
                            ascending=[True],
                            ignore_index=True)

output.to_excel('./test_page.xlsx', index=False)

output

,No,Url,SKU Count,Page
0,1,https://www.dormanproducts.com/gsearch.aspx?ty...,1212,13


In [10]:
crawler_status = 'ok'

crawler_status

'ok'

In [11]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [12]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[False],
                                            ignore_index=True)
    output_error.to_excel('./test_page_error.xlsx', index=False)

output_error

""
